In [ ]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# pip install Sastrawi

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import gensim
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import gensim
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Membaca file CSV
df = pd.read_csv('Training_Testing_Data16444.csv')

# # Membagi data menjadi data training dan data testing
# train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
# # test_size = 0.2 berarti 20% data akan digunakan untuk testing, sementara 80% digunakan untuk training
# # random_state digunakan untuk mengacak data

# # Menampilkan informasi jumlah data
# print('Jumlah data training:', len(train_data))
# print('Jumlah data testing:', len(test_data))


In [ ]:
# train_data, test_data = train_test_split(df, test_size=0.25, random_state=42)


In [ ]:
# train_data

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# #Train Data

# print("Data shape:",df)

# Emosi1Df = df[df["Label"]=="Gembira"].drop("Label",axis=1)
# Emosi1Df["Label"] = 0

# Emosi2Df = df[df["Label"]=="Sedih"].drop("Label",axis=1)
# Emosi2Df["Label"] = 1

# Emosi3Df = df[df["Label"]=="Muak"].drop("Label",axis=1)
# Emosi3Df["Label"] = 2

# Emosi4Df = df[df["Label"]=="Marah"].drop("Label",axis=1)
# Emosi4Df["Label"] = 3

# Emosi5Df = df[df["Label"]=="Takut"].drop("Label",axis=1)
# Emosi5Df["Label"] = 4

# Emosi6Df = df[df["Label"]=="Terkejut"].drop("Label",axis=1)
# Emosi6Df["Label"] = 5

# df = pd.concat([Emosi1Df,Emosi2Df,Emosi3Df,Emosi4Df,Emosi5Df,Emosi6Df],ignore_index=True)
# print("Final total dataset:",df.shape)

In [ ]:
# train_data

In [ ]:
# # Simpan hasil ke file CSV
# df.to_csv('output.csv', index=False)

In [ ]:
# # Membaca file CSV
# df = pd.read_csv('output (1).csv')

In [ ]:


# # Menggunakan tokenizer dari NLTK
# df['text'] = df['text'].apply(word_tokenize)

# # Menampilkan DataFrame dengan teks yang telah di-tokenisasi
# print(df)


In [ ]:


# Assuming 'text' is the column containing the text data in your DataFrame
list_of_sent = [word_tokenize(str(sent)) for sent in df["text"]]
w2v_model = gensim.models.Word2Vec(list_of_sent,vector_size=200,window=3,min_count=2,sg=1,negative=10, ns_exponent=0.75, workers=4,epochs=10)#Skipgram sg=1
w2v_words = list(w2v_model.wv.index_to_key)


In [ ]:
# Save the Word2Vec model
model_path = "model_sg.bin"  # Ganti dengan path tempat Anda ingin menyimpan model
w2v_model.save(model_path)


In [ ]:
len(w2v_words)

30955

In [ ]:
#Vectorize train text data
listof_sent_vec=[]
for sent in tqdm(list_of_sent):
    sent_vec = np.zeros(200)
    cnt_words =0;
    for word in sent:
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    listof_sent_vec.append(sent_vec)

Label = df["Label"]
list_col=tuple(range(200))
Scaler = StandardScaler()
data_vec = Scaler.fit_transform(listof_sent_vec)
W2v_data=pd.DataFrame(data=data_vec, columns=list_col)
W2v_data["Label"] = Label
print(W2v_data.head(10))
print(W2v_data.shape)

100%|██████████| 98664/98664 [01:08<00:00, 1445.35it/s]


          0         1         2         3         4         5         6  \
0  1.242961 -0.643272  1.018399  0.715394 -0.270445 -1.324530  0.869032   
1 -1.801012 -1.158490  0.033325 -0.190662  0.645363  0.242590  0.910903   
2 -0.914484  1.193085 -0.941452 -0.640334  0.087640  0.942915  0.540623   
3  0.786590  0.578556  1.001725 -0.433143 -0.025089  0.037009  0.041258   
4 -0.120129 -1.394775  1.246648 -1.373574  0.220639 -0.348208 -0.548035   
5  0.166675 -0.098405  0.371948 -0.162663  0.196284 -0.024860 -0.163535   
6 -0.303381  0.746830 -0.442223 -1.201420  0.337949  1.564170 -0.172779   
7  0.455673  0.991878 -0.079048 -0.317153 -0.249131  0.411229  1.118792   
8  0.500198 -1.144849  0.640001 -0.373842 -0.014655 -1.023570 -1.383252   
9 -1.732755  0.222601 -0.395225 -0.780365  0.641238  0.970716  0.098041   

          7         8         9  ...       191       192       193       194  \
0 -0.494344 -0.244893  0.513347  ...  0.606476 -0.312372 -0.456044 -1.566594   
1 -0.516855 -0

In [ ]:

X = W2v_data.drop("Label",axis=1).to_numpy()
X

array([[ 1.24296074, -0.64327217,  1.0183992 , ...,  0.80669981,
        -0.37492206,  0.17452345],
       [-1.80101167, -1.15848953,  0.03332522, ..., -0.88330463,
        -1.01072072, -0.09254181],
       [-0.91448441,  1.19308497, -0.9414521 , ...,  0.65612795,
        -0.68866175, -0.9236795 ],
       ...,
       [ 0.92531608, -1.94159433,  1.7033243 , ..., -0.46898184,
        -0.3182432 ,  1.34929006],
       [ 0.57416666,  0.29251189,  0.07056491, ...,  0.46071642,
         1.00639554, -0.04385427],
       [ 1.33838942, -0.08529013,  0.3053093 , ..., -0.99143742,
         0.83502975, -1.43788548]])

In [ ]:
# Load the Word2Vec model
from gensim.models import Word2Vec
model_path = "model_sg.bin"  # Replace with the path where you saved the model
w2v_model = Word2Vec.load(model_path)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'label' is the column containing the corresponding labels in your DataFrame
labels = df['Label'].tolist()

# Initialize label encoder
label_encoder = LabelEncoder()

# Encode labels to integers
encoded_labels = label_encoder.fit_transform(labels)

In [ ]:
y = W2v_data["Label"].to_numpy()
y[0:5]

array(['Sedih', 'Sedih', 'Sedih', 'Sedih', 'Sedih'], dtype=object)

In [ ]:
pip install --upgrade keras tensorflow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.datasets import imdb

#import the required library

from sklearn.preprocessing import LabelEncoder
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense,MaxPooling1D
from keras.layers import LSTM, Flatten, Dropout, Conv1D, Input
from tensorflow.keras.layers import Embedding
from keras.preprocessing import sequence
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Assuming 'text' is the column containing the text data in your DataFrame
texts = df['text'].tolist()

# Initialize the tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the texts
tokenizer.fit_on_texts(texts)

# Limit the tokenizer vocabulary to the top n words
top_n = 10000  # Set the desired number of top words
tokenizer.num_words = top_n

# Convert texts to sequences of integers
sequences = tokenizer.texts_to_sequences(texts)


In [ ]:
# Hitung panjang maksimum dari sequences
max_length = max(len(seq) for seq in sequences)
print("Maximum sequence length:", max_length)


Maximum sequence length: 250


In [ ]:
# Assuming 'label' is the column containing the corresponding labels in your DataFrame
labels = df['Label'].tolist()

# Pad sequences to a fixed length
max_sequence_length = max_length  # Set the maximum sequence length based on the shape of X_train
# Set the desired maximum sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.25, random_state=42)

# Split the training data further into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Print the vocabulary size
vocab_size = min(top_n, len(tokenizer.word_index)) + 1
print("Vocabulary size:", vocab_size)


Vocabulary size: 10001


In [ ]:
# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(labels)
y_train = label_encoder.transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

# Convert labels to categorical representation
num_classes = len(label_encoder.classes_)
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
# decoded_texts = tokenizer.sequences_to_texts(sequences)
# decoded_labels = np.array(labels)

# # Print the decoded texts and labels
# print(decoded_texts)
# print(decoded_labels)


In [ ]:
print("Shape of train data:", X_train.shape)
print("Shape of Test data:", X_test.shape)
print("Shape of CV data:", X_val.shape)

Shape of train data: (59198, 250)
Shape of Test data: (24666, 250)
Shape of CV data: (14800, 250)


In [ ]:
# # Convert labels to categorical representation
# label_encoder = LabelEncoder()
# y_train = label_encoder.fit_transform(y_train)
# y_val = label_encoder.transform(y_val)
# y_test = label_encoder.transform(y_test)

In [ ]:
max_review_length = 250
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
X_cv = sequence.pad_sequences(X_val, maxlen=max_review_length)


In [ ]:
y_train[0:5]

array([2, 5, 4, 4, 0])

In [ ]:
# decoded_texts = tokenizer.sequences_to_texts(sequences)
# decoded_labels = np.array(labels)

# # Print the decoded texts and labels
# print(decoded_texts)
# print(decoded_labels)


In [ ]:
from tensorflow.keras.layers import Embedding, Conv1D, AveragePooling1D, GlobalAveragePooling1D, concatenate, Dense, Dropout, Flatten, LSTM, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import sequence
import numpy as np
from tensorflow.keras import regularizers
import tensorflow as tf

# Load the Word2Vec model
model_path = "model_sg.bin"  # Replace with the path where you saved the model
w2v_model = Word2Vec.load(model_path)

# Get the word vectors
word_vectors = w2v_model.wv

# Create the embedding matrix
embedding_vector_length = 200  # Adjust according to the dimension of the word vectors in your Word2Vec model
embedding_matrix = np.zeros((vocab_size, embedding_vector_length))
for word, index in tokenizer.word_index.items():
    if index < vocab_size and word in word_vectors:
        embedding_matrix[index] = word_vectors[word]

In [ ]:
# Define the model architecture
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_vector_length, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False)(input_layer)

# CNN Layers
cnn_layer1 = Conv1D(128, kernel_size=3, activation='relu')(embedding_layer)
cnn_layer1 = AveragePooling1D(pool_size=2)(cnn_layer1)
cnn_layer2 = Conv1D(128, kernel_size=4, activation='relu')(embedding_layer)
cnn_layer2 = AveragePooling1D(pool_size=2)(cnn_layer2)
cnn_layer3 = Conv1D(128, kernel_size=5, activation='relu')(embedding_layer)
cnn_layer3 = AveragePooling1D(pool_size=2)(cnn_layer3)
cnn_layer1 = GlobalAveragePooling1D()(cnn_layer1)  # Adjusted
cnn_layer2 = GlobalAveragePooling1D()(cnn_layer2)  # Adjusted
cnn_layer3 = GlobalAveragePooling1D()(cnn_layer3)  # Adjusted
cnn_layer = concatenate([cnn_layer1, cnn_layer2, cnn_layer3])
cnn_layer = Dropout(0.5)(cnn_layer)
cnn_layer = Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01))(cnn_layer)
cnn_layer = Dropout(0.5)(cnn_layer)

# LSTM Layer
lstm_layer = Embedding(input_dim=vocab_size, output_dim=embedding_vector_length, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False)(input_layer)
lstm_layer = LSTM(64)(lstm_layer)  # Modified to a single LSTM layer
lstm_layer = Dropout(0.5)(lstm_layer)

# Embedding Layer
embedding_only_layer = Flatten()(embedding_layer)
embedding_only_layer = Dropout(0.5)(embedding_only_layer)
embedding_only_layer = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(embedding_only_layer)
embedding_only_layer = Dropout(0.5)(embedding_only_layer)

# Concatenate all layers
merged_layer = concatenate([cnn_layer, lstm_layer, embedding_only_layer])
merged_layer = Dropout(0.5)(merged_layer)
output_layer = Dense(6, activation='softmax')(merged_layer)



In [ ]:
# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Create the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile the model with categorical crossentropy loss
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 250)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 250, 200)     2000200     ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 248, 128)     76928       ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 247, 128)     102528      ['embedding[0][0]']              
                                                                                              

In [ ]:
# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.25, random_state=42)

# Split the training data further into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Print the shapes of the data splits
print("Shape of train data:", X_train.shape)
print("Shape of validation data:", X_val.shape)
print("Shape of test data:", X_test.shape)
print("Shape of train labels:", len(y_train))
print("Shape of validation labels:", len(y_val))
print("Shape of test labels:", len(y_test))


Shape of train data: (59198, 250)
Shape of validation data: (14800, 250)
Shape of test data: (24666, 250)
Shape of train labels: 59198
Shape of validation labels: 14800
Shape of test labels: 24666


In [ ]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

import numpy as np

# Convert labels to numpy arrays
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

batch_size = 64  # Increase the batch size if memory allows
epochs = 100  # Increase the number of epochs

# Train the model with early stopping
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                    validation_data=(X_val, y_val), verbose=1,
                    callbacks=[early_stopping])

# Evaluate the model on training and test data
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Train Loss: {train_loss:.4f}")
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



Epoch 1/100
925/925 [==============================] - 509s 548ms/step - loss: 1.2795 - accuracy: 0.5891 - val_loss: 0.8074 - val_accuracy: 0.7407
Epoch 2/100
925/925 [==============================] - 503s 544ms/step - loss: 0.7924 - accuracy: 0.7358 - val_loss: 0.6149 - val_accuracy: 0.7828
Epoch 3/100
925/925 [==============================] - 542s 586ms/step - loss: 0.6684 - accuracy: 0.7695 - val_loss: 0.5559 - val_accuracy: 0.7989
Epoch 4/100
925/925 [==============================] - 503s 544ms/step - loss: 0.6130 - accuracy: 0.7887 - val_loss: 0.5219 - val_accuracy: 0.8146
Epoch 5/100
925/925 [==============================] - 502s 543ms/step - loss: 0.5787 - accuracy: 0.7981 - val_loss: 0.4956 - val_accuracy: 0.8214
Epoch 6/100
925/925 [==============================] - 503s 543ms/step - loss: 0.5440 - accuracy: 0.8105 - val_loss: 0.4781 - val_accuracy: 0.8242
Epoch 7/100
925/925 [==============================] - 502s 543ms/step - loss: 0.5246 - accuracy: 0.8172 - val_loss: 0

In [ ]:

val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)


print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")


Validation Loss: 0.5323
Validation Accuracy: 0.8132
